In [0]:
from pyspark.sql.functions import col, sum, count, avg, desc, round

nf_silver = spark.table("classes.silver.notas_fiscais")


gold_mensal = nf_silver.groupBy("invoice_year", "invoice_month") \
    .agg(
        sum("total_line_value").alias("faturamento_bruto"),
        sum("tax_amount").alias("total_impostos"),
        count("invoice_id").alias("qtde_notas_emitidas"),
        sum("product_quantity").alias("total_itens_vendidos")
    ) \
    .orderBy(col("invoice_year").desc(), col("invoice_month").desc())


gold_mensal = gold_mensal.withColumn("faturamento_bruto", round(col("faturamento_bruto"), 2)) \
                         .withColumn("total_impostos", round(col("total_impostos"), 2))


gold_vendedores = nf_silver.groupBy("salesperson_name") \
    .agg(
        sum("total_line_value").alias("valor_total_vendas"),
        count("invoice_id").alias("total_vendas_feitas"),
        avg("total_line_value").alias("ticket_medio")
    ) \
    .orderBy(col("valor_total_vendas").desc())


gold_vendedores = gold_vendedores.withColumn("valor_total_vendas", round(col("valor_total_vendas"), 2)) \
                                 .withColumn("ticket_medio", round(col("ticket_medio"), 2))


print("--- Visão Mensal (Gold) ---")
display(gold_mensal)

print("--- Ranking Vendedores (Gold) ---")
display(gold_vendedores)

In [0]:

df_faturamento = spark.sql("""
    SELECT 
        concat(invoice_year, '-', invoice_month) as periodo, 
        faturamento_bruto,
        total_impostos
    FROM classes.gold.faturamento_mensal 
    ORDER BY invoice_year, invoice_month
""")

display(df_faturamento)

Databricks visualization. Run in Databricks to view.

In [0]:

df_ranking = spark.sql("""
    SELECT 
        salesperson_name, 
        valor_total_vendas 
    FROM classes.gold.performance_vendedores 
    ORDER BY valor_total_vendas DESC 
    LIMIT 10
""")

display(df_ranking)

Databricks visualization. Run in Databricks to view.

In [0]:

gold_mensal.write.mode("overwrite").saveAsTable("classes.gold.faturamento_mensal")


gold_vendedores.write.mode("overwrite").saveAsTable("classes.gold.performance_vendedores")